# 05 Portfolio Optimization
- Use forecasted and historical returns
- Efficient Frontier, Sharpe Ratio
- Portfolio selection and justification

# 05 Portfolio Optimization
This notebook demonstrates how to use forecasted and historical returns to optimize a portfolio using Modern Portfolio Theory (MPT) and PyPortfolioOpt.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting

In [ ]:
# Load processed price data for TSLA, BND, SPY
assets = ['TSLA', 'BND', 'SPY']
dfs = []
for asset in assets:
    df = pd.read_csv(f'../data/processed/{asset}_processed.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    dfs.append(df['Adj Close'])
prices = pd.concat(dfs, axis=1)
prices.columns = assets
prices = prices.dropna()
prices.head()

In [ ]:
# Calculate expected returns and sample covariance matrix
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)
print('Expected annual returns:')
print(mu)
print('\nCovariance matrix:')
print(S)

In [ ]:
# Run portfolio optimization for maximum Sharpe ratio and minimum volatility
# Efficient Frontier
ef = EfficientFrontier(mu, S)
max_sharpe_weights = ef.max_sharpe()
cleaned_max_sharpe = ef.clean_weights()
perf_max_sharpe = ef.portfolio_performance(verbose=True)

# Minimum volatility portfolio
ef_min = EfficientFrontier(mu, S)
ef_min.min_volatility()
cleaned_min_vol = ef_min.clean_weights()
perf_min_vol = ef_min.portfolio_performance(verbose=True)

print('Max Sharpe Weights:', cleaned_max_sharpe)
print('Min Volatility Weights:', cleaned_min_vol)

In [ ]:
# Plot the efficient frontier and mark optimal portfolios
fig, ax = plt.subplots(figsize=(8,6))
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)

# Mark the max Sharpe and min volatility portfolios
from pypfopt import discrete_allocation

# Get risk and return for max Sharpe and min vol
ret_sharpe, vol_sharpe, _ = perf_max_sharpe
ret_min, vol_min, _ = perf_min_vol
ax.scatter(vol_sharpe, ret_sharpe, marker='*', color='r', s=200, label='Max Sharpe')
ax.scatter(vol_min, ret_min, marker='o', color='b', s=200, label='Min Volatility')
plt.title('Efficient Frontier with Optimal Portfolios')
plt.xlabel('Volatility (Risk)')
plt.ylabel('Return')
plt.legend()
plt.show()

In [ ]:
# Summarize and recommend an optimal portfolio
print('Recommended Portfolio (Max Sharpe):')
for asset, weight in cleaned_max_sharpe.items():
    print(f'{asset}: {weight:.2%}')
print(f'Expected annual return: {perf_max_sharpe[0]:.2%}')
print(f'Annual volatility: {perf_max_sharpe[1]:.2%}')
print(f'Sharpe Ratio: {perf_max_sharpe[2]:.2f}')

## Discussion & Justification
- Justify the choice of optimal portfolio (e.g., max Sharpe for risk-adjusted return, min volatility for safety).
- Summarize the weights, expected return, volatility, and Sharpe ratio.
- Discuss how the forecasted TSLA return influenced the allocation.